# Database Tests

1\. Test user

In [1]:
-- Test User Creation and Management
DECLARE @TestUserID INT;
DECLARE @TestEmail NVARCHAR(255) = 'test.user@example.com';

-- Create test user
INSERT INTO recycling.Users (
    Email, 
    PhoneNumber, 
    FirstName, 
    LastName, 
    DefaultAddress,
    DefaultLatitude,
    DefaultLongitude
)
VALUES (
    @TestEmail,
    '+1234567890',
    'Test',
    'User',
    '123 Test Street, Test City',
    -1.2921,
    36.8219
);

SET @TestUserID = SCOPE_IDENTITY();
PRINT 'Created test user with ID: ' + CAST(@TestUserID AS NVARCHAR(10));

-- Verify user creation
IF EXISTS (SELECT 1 FROM recycling.Users WHERE UserID = @TestUserID)
    PRINT 'User creation test: PASSED'
ELSE
    THROW 50000, 'User creation test: FAILED', 1; 

(1 row affected)

Created test user with ID: 1

User creation test: PASSED

Total execution time: 00:00:00.044

2\. Test business

In [2]:
-- Test Business Creation
DECLARE @TestBusinessID INT;

INSERT INTO recycling.Businesses (
    Name,
    FormattedAddress,
    Latitude,
    Longitude,
    PhoneNumber,
    Website
)
VALUES (
    'Test Recycling Company',
    '456 Recycling Ave, Green City',
    -1.2921,
    36.8219,
    '+254123456789',
    'www.testrecycling.com'
);

SET @TestBusinessID = SCOPE_IDENTITY();
PRINT 'Created test business with ID: ' + CAST(@TestBusinessID AS NVARCHAR(10));

-- Verify business creation
IF EXISTS (SELECT 1 FROM recycling.Businesses WHERE BusinessID = @TestBusinessID)
    PRINT 'Business creation test: PASSED'
ELSE
    THROW 50000, 'Business creation test: FAILED', 1; 

(1 row affected)

Created test business with ID: 1

Business creation test: PASSED

Total execution time: 00:00:00.033

3\. Test subscription

In [6]:
-- Test User Subscription
DECLARE @TestSubscriptionID INT;

INSERT INTO recycling.UserSubscriptions (
    UserID,
    BusinessID,
    NotificationsEnabled
)
VALUES (
    1,
    1,
    1
);

SET @TestSubscriptionID = SCOPE_IDENTITY();
PRINT 'Created test subscription with ID: ' + CAST(@TestSubscriptionID AS NVARCHAR(10));

-- Verify subscription
IF EXISTS (SELECT 1 FROM recycling.UserSubscriptions WHERE SubscriptionID = @TestSubscriptionID)
    PRINT 'Subscription creation test: PASSED'
ELSE
    THROW 50000, 'Subscription creation test: FAILED', 1; 

(1 row affected)

Created test subscription with ID: 1

Subscription creation test: PASSED

Total execution time: 00:00:00.037

4\. Test transaction

In [9]:
-- Test Recycling Transaction
DECLARE @TestTransactionID INT;
DECLARE @MaterialsJSON NVARCHAR(MAX) = '[
    {
        "materialCategoryID": 1,
        "quantity": 10.5,
        "unit": "kg",
        "pricePerUnit": 20.00
    },
    {
        "materialCategoryID": 2,
        "quantity": 5.0,
        "unit": "kg",
        "pricePerUnit": 15.00
    }
]';

-- Execute transaction stored procedure
EXEC recycling.usp_RecordRecyclingTransaction
    @BusinessID = 1,
    @UserID = 1,
    @TransactionType = 'PURCHASE',
    @PaymentAmount = 285.00,  -- (10.5 * 20.00) + (5.0 * 15.00)
    @PaymentCurrency = 'KES',
    @Notes = 'Test transaction',
    @Materials = @MaterialsJSON;

-- Verify transaction
SELECT @TestTransactionID = MAX(TransactionID) 
FROM recycling.MaterialTransaction
WHERE BusinessID = 1 AND UserID = 1;

IF EXISTS (
    SELECT 1 
    FROM recycling.RecyclingTransactions 
    WHERE TransactionID = @TestTransactionID
)
    PRINT 'Transaction creation test: PASSED'
ELSE
    THROW 50000, 'Transaction creation test: FAILED', 1;

-- Verify transaction materials
IF EXISTS (
    SELECT 1 
    FROM recycling.TransactionMaterials 
    WHERE TransactionID = @TestTransactionID
)
    PRINT 'Transaction materials test: PASSED'
ELSE
    THROW 50000, 'Transaction materials test: FAILED', 1; 

: Msg 208, Level 16, State 1, Procedure recycling.usp_RecordRecyclingTransaction, Line 19
Invalid object name 'recycling.RecyclingTransactions'.

: Msg 266, Level 16, State 2, Procedure recycling.usp_RecordRecyclingTransaction, Line 19
Transaction count after EXECUTE indicates a mismatching number of BEGIN and COMMIT statements. Previous count = 0, current count = 1.

: Msg 208, Level 16, State 1, Line 29
Invalid object name 'recycling.RecyclingTransactions'.

Total execution time: 00:00:00.032